In [1]:
print('Paskaita nr. 33\n2024-09-25')

import datetime
print('Pasiruošta',datetime.datetime.now())

import mysql.connector as cnt
import sqlite3

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Paskaita nr. 33
2024-09-25
Pasiruošta 2024-09-25 21:06:08.382340


In [2]:
# atrinkti 10 populiariausių gamintojų iš autopliuslt, 
# pateikti tokius stulpelius - gamintojas, vidutinė kaina, vidutinė rida, vidutinis amžius, modelių kiekis. 
# Gautus rezultatus surašyti į SQLite duomebazę, pavadinimu "Testas.db", lentelės pavadinimas kokį sugalvosite. 
# Viską turi atlikti python kodas. 
# PANDAS NEGALIMA NAUDOTI, viską atlikite per cursor() objektus.

# Bandymai:

# DB = cnt.connect(
#     host = '88.223.121.130',
#     user='user',
#     password='slaptazodis',
#     port=3306,
#     database='works'
# )
# C=DB.cursor()

# SQLite naudojimas:

# import sqlite3

# sdb = sqlite3.connect('Testas.db')
# cs = sdb.cursor()

# sql = '''create table if not exists pop_auto_gamintojai
# (
#     name text not null,
#     surname text not null,
#     age integer not null,
#     salary real not null
# )
# '''
# cs.execute(sql)

# sql_template = '''insert into T1 values (?,?,?,?)'''
# data = [("Onynė","Mažutytė",18,1860),("Jonas","Varnas",28,1670),("Petras","Repšys",42,2760)]

# cs.executemany(sql_template,data)
# sdb.commit()

In [3]:
# Užduoties pirmoji dalis - duomenų gavimas:
# atrinkti 10 populiariausių gamintojų iš autopliuslt, 
# pateikti tokius stulpelius - gamintojas, vidutinė kaina, vidutinė rida, vidutinis amžius, modelių kiekis. 

DB = cnt.connect(
    host = '88.223.121.130',
    user='user',
    password='slaptazodis',
    port=3306,
    database='works'
)
C=DB.cursor()

sql="""
    select
        G, 
        cast(avg(P) as float), 
        cast(avg(R) as float), 
        cast(avg(A) as float), 
        count(*) as C 
    from (select 
        gamintojas as G, 
        cast(replace(price,' ','') as int) as P,
        cast(replace(rida, ' ','') as int) as R, 
        2024-cast(substring(pagaminimo_data,1,4) as int) as A
    from autopliuslt) as T1
    group by G
    order by C desc limit 10;
"""
C.execute(sql)
ans = C.fetchall()

print('Paimta top 10 iš db:')
print(ans)


Paimta top 10 iš db:
[('BMW', 10652.1, 260654.0, 15.2219, 365), ('Volkswagen', 6190.71, 237651.0, 15.7903, 310), ('Audi', 8447.67, 241376.0, 16.7665, 227), ('Mercedes-Benz', 15080.3, 261259.0, 15.1864, 177), ('Toyota', 8712.73, 201755.0, 14.6039, 154), ('Volvo', 7847.71, 278037.0, 15.3116, 138), ('Opel', 4005.84, 226788.0, 16.3209, 134), ('Ford', 5592.4, 225685.0, 14.8818, 110), ('Peugeot', 6748.5, 205881.0, 13.4103, 78), ('Skoda', 10808.3, 181196.0, 10.7703, 74)]


In [4]:
# Užduoties antroji dalis - duomenų įrašymas:
# Pateikti tokius stulpelius - gamintojas, vidutinė kaina, vidutinė rida, vidutinis amžius, modelių kiekis. 
# Gautus rezultatus surašyti į SQLite duomebazę, pavadinimu "Testas.db", lentelės pavadinimas kokį sugalvosite. 
# Viską turi atlikti python kodas. 

sdb = sqlite3.connect('Testas.db')
cs = sdb.cursor()

sql_table = '''
create table if not exists top_auto
(
    gamintojas text not null,
    kaina real not null,
    rida real not null,
    amz real not null,
    n real not null
)
'''
cs.execute(sql_table)

sql_template = '''insert into top_auto values (?,?,?,?,?)'''
cs.executemany(sql_template, ans)

sdb.commit()
sdb.close()

In [5]:
# JSON duomenų gavimas:

# Eismo įvykių, įvykusių Lietuvos Respublikoje, kurių metu žuvo ar buvo sužeisti žmonės, duomenys:
# https://data.gov.lt/datasets/509/

# nuskaitykite JSON failą -  2023 metus

# konvertuokite dataLaikas stulpelį į datetime
# pd.to_datetime(df[stulpelis])

# pridėkite naują stulpelį su paros laikais (rytas, vakaras ...)
# pridėkite naują stulplelį, kur būtų tik įvykio valanda
# Vizualizuokite, kuriuo paros laiku, kurią valandą įvyksta daugiausiai eismo įvykių
 
# sekantiems klausimams patys sugalvojate tinkamiausias vizualizacijas
# Vyrų ar moterų buvo daugiau, sukėlusių eismo įvykį?
# koks vidutinis kaltininkų amžius?
# Kokiomis baudų rūšimis buvo nubausti vyrai/moterys (administracinė, baudžiamoji) (teisenosStadijaBusena)

import pandas as pd

df = pd.read_json('../../DATA/ei_2023_12_31.json')
# print(df.head(3))

In [10]:
# intervalai: # 0-6 - naktis, 7-10 - rytas, 11-12 - priešpietė, 
# 13-14-pietūs, 15-18-pavakarė, 19-22 - vakaras, 23-0 - naktis.
# Parašykite funkciją, kuri iš times sąrašo verčių sugeneruotų tekstų (rytas, vakaras, popietė ... ) sąrašą.
# Pritaikykite map(). Suskaičiuokite, kiek kartų pasikartojo atitinkami tekstai, 
    
def laikoPavadinimas2(laikas):
    laik=laikas.hour
    if laik<0 :
        return 'klaida: 001'
    elif laik<7 :
        return 'naktis'
    elif laik<11 :
        return 'rytas'
    elif laik<13 :
        return 'priešpietė'
    elif laik<15 :
        return 'pietūs'
    elif laik<19 :
        return 'pavakarė'
    elif laik<23 :
        return 'vakaras'
    elif laik<24 :
        return 'naktis'
    else :
        return 'klaida: 002'


In [17]:
df['dataLaikas'] = pd.to_datetime(df['dataLaikas'])
# print(df.info())

# df['laikoPavadinimas'] = laikoPavadinimas(df['dataLaikas'])
print(list(map(laikoPavadinimas2,df['dataLaikas'])))

df['laikoPavadinimas'] = list(map(laikoPavadinimas2,df['dataLaikas']))
df['valanda'] = df['dataLaikas'].dt.hour

['naktis', 'naktis', 'naktis', 'naktis', 'naktis', 'naktis', 'naktis', 'naktis', 'naktis', 'naktis', 'naktis', 'naktis', 'naktis', 'rytas', 'naktis', 'priešpietė', 'vakaras', 'pietūs', 'priešpietė', 'pavakarė', 'pavakarė', 'vakaras', 'pavakarė', 'pavakarė', 'pavakarė', 'pietūs', 'naktis', 'pavakarė', 'pavakarė', 'pavakarė', 'pavakarė', 'pavakarė', 'pavakarė', 'pavakarė', 'pavakarė', 'pavakarė', 'vakaras', 'vakaras', 'priešpietė', 'vakaras', 'pavakarė', 'naktis', 'pavakarė', 'vakaras', 'naktis', 'naktis', 'naktis', 'naktis', 'naktis', 'rytas', 'rytas', 'pavakarė', 'rytas', 'rytas', 'rytas', 'rytas', 'pavakarė', 'rytas', 'rytas', 'pavakarė', 'pietūs', 'priešpietė', 'rytas', 'pietūs', 'rytas', 'pietūs', 'rytas', 'rytas', 'pavakarė', 'rytas', 'priešpietė', 'vakaras', 'priešpietė', 'vakaras', 'pietūs', 'pietūs', 'priešpietė', 'rytas', 'pavakarė', 'pietūs', 'pietūs', 'vakaras', 'vakaras', 'pavakarė', 'pavakarė', 'naktis', 'pietūs', 'pietūs', 'pavakarė', 'pietūs', 'pavakarė', 'pavakarė', 'pav

In [21]:
# print(df.head(3))
# print(df.info())

In [53]:
# print(df['eismoDalyviai'][0][0]['kaltininkas'])

for i in range(3111,3119):
    eismoĮvykioDalyviai=df['eismoDalyviai'][i]
    for d in eismoĮvykioDalyviai:
        print( f" i={i} dalyvisId={d['dalyvisId']}, lytis={d['lytis']}, kategorija={d['kategorija']}, kaltininkas={d['kaltininkas']}" )
    print()

# print(eismoĮvykioDalyviai)

 i=3111 dalyvisId=48655555, lytis=Vyras, kategorija=Automobilio vairuotojas, kaltininkas=Taip
 i=3111 dalyvisId=48655557, lytis=None, kategorija=Automobilio vairuotojas, kaltininkas=Ne
 i=3111 dalyvisId=48655553, lytis=Vyras, kategorija=Automobilio vairuotojas, kaltininkas=Ne
 i=3111 dalyvisId=48655556, lytis=Moteris, kategorija=Automobilio vairuotojas, kaltininkas=Ne
 i=3111 dalyvisId=48655554, lytis=Vyras, kategorija=Automobilio vairuotojas, kaltininkas=Taip

 i=3112 dalyvisId=48615944, lytis=Vyras, kategorija=Automobilio vairuotojas, kaltininkas=Ne
 i=3112 dalyvisId=48615945, lytis=Vyras, kategorija=Automobilio vairuotojas, kaltininkas=Taip

 i=3113 dalyvisId=48925696, lytis=Moteris, kategorija=Keleivis, kaltininkas=Ne
 i=3113 dalyvisId=48925694, lytis=Moteris, kategorija=Keleivis, kaltininkas=Ne
 i=3113 dalyvisId=48925693, lytis=Moteris, kategorija=Automobilio vairuotojas, kaltininkas=Ne
 i=3113 dalyvisId=48925695, lytis=Vyras, kategorija=Automobilio vairuotojas, kaltininkas=Taip



In [62]:
print('Iš viso fiksuota įvykių:',len(df['eismoDalyviai']))
print()
print('Tarp jų - keleiviai kaltininkai:')
nr=0
for i in range(len(df['eismoDalyviai'])):
    eismoĮvykioDalyviai=df['eismoDalyviai'][i]
    for d in eismoĮvykioDalyviai:
        if d['kategorija']=='Keleivis' and d['kaltininkas']=='Taip':
            nr=nr+1
            print( f" i={nr} dalyvisId={d['dalyvisId']}, lytis={d['lytis']}, kategorija={d['kategorija']}, kaltininkas={d['kaltininkas']}" )


Iš viso fiksuota įvykių: 24240

Tarp jų - keleiviai kaltininkai:
 i=1 dalyvisId=48572477, lytis=Moteris, kategorija=Keleivis, kaltininkas=Taip
 i=2 dalyvisId=48616782, lytis=Moteris, kategorija=Keleivis, kaltininkas=Taip
 i=3 dalyvisId=48606488, lytis=Vyras, kategorija=Keleivis, kaltininkas=Taip
 i=4 dalyvisId=48787001, lytis=Moteris, kategorija=Keleivis, kaltininkas=Taip
 i=5 dalyvisId=48666950, lytis=Vyras, kategorija=Keleivis, kaltininkas=Taip
 i=6 dalyvisId=48870675, lytis=Moteris, kategorija=Keleivis, kaltininkas=Taip
 i=7 dalyvisId=48680951, lytis=Moteris, kategorija=Keleivis, kaltininkas=Taip
 i=8 dalyvisId=48800694, lytis=Vyras, kategorija=Keleivis, kaltininkas=Taip
 i=9 dalyvisId=48682898, lytis=Moteris, kategorija=Keleivis, kaltininkas=Taip
 i=10 dalyvisId=48757545, lytis=Vyras, kategorija=Keleivis, kaltininkas=Taip
 i=11 dalyvisId=48725684, lytis=Vyras, kategorija=Keleivis, kaltininkas=Taip
 i=12 dalyvisId=48737687, lytis=Vyras, kategorija=Keleivis, kaltininkas=Taip
 i=13 da